# Creating a molecule "explorer"

In this notebook, I use [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/) to create an interactive molecule explorer. This can, for instance, be useful if we have a data set containing a lot of molecules and we want to visualize the molecules.

In [ ]:
import pandas as pd
from io import StringIO
from rdkit import Chem
from rdkit.Chem import (
    AllChem,
    rdCoordGen,
)
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.ipython_useSVG = True  # Use higher quality images for molecules
import py3Dmol
from ipywidgets import interact, Dropdown

In [ ]:
data_set = StringIO(
    """names,smiles
Amigdalin,N#CC(OC1OC(COC2OC(CO)C(O)C(O)C2O)C(O)C(O)C1O)c1ccccc1
Fenfuram,Cc1occc1C(=O)Nc1ccccc1
citral,CC(C)=CCCC(C)=CC=O
Picene,c1ccc2c(c1)ccc1c2ccc2c3ccccc3ccc21
Thiophene,c1ccsc1
benzothiazole,c1ccc2scnc2c1
"2,2,4,6,6'-PCB",Clc1cc(Cl)c(-c2c(Cl)cccc2Cl)c(Cl)c1
Estradiol,CC12CCC3c4ccc(O)cc4CCC3C1CCC2O
Dieldrin,ClC1=C(Cl)C2(Cl)C3C4CC(C5OC45)C3C1(Cl)C2(Cl)Cl
Rotenone,C=C(C)C1Cc2c(ccc3c2OC2COc4cc(OC)c(OC)cc4C2C3=O)O1
2-pyrrolidone,O=C1CCCN1
2-Chloronapthalene,Clc1ccc2ccccc2c1
1-Pentene ,C=CCCC
Primidone,CCC1(c2ccccc2)C(=O)NCNC1=O
Tetradecane,CCCCCCCCCCCCCC
2-Chloropropane,CC(C)Cl
2-Methylbutanol,CCC(C)CO
Benzonitrile,N#Cc1ccccc1
Diazinon,CCOP(=S)(OCC)Oc1cc(C)nc(C(C)C)n1
2-Undecanol,CCCCCCCCCC(C)O"""
)

In [ ]:
table = pd.read_csv(data_set)
table

In [ ]:
def molecule_to_3d(molecule):
    """Add 3D coordinates for RDKit molecules."""
    mol = Chem.Mol(molecule)
    mol = AllChem.AddHs(mol, addCoords=True)
    AllChem.EmbedMolecule(mol)
    AllChem.MMFFOptimizeMolecule(mol)
    return mol


def molecules_from_smiles(smiles):
    """Generate RDKit molecules from smiles."""
    molecules = []
    for smilei in smiles:
        mol = Chem.MolFromSmiles(smilei)
        rdCoordGen.AddCoords(mol)
        molecules.append(mol)
    return molecules

In [ ]:
names = table["names"].values
smiles = table["smiles"].values

molecules = molecules_from_smiles(smiles)
molecules_3d = [molecule_to_3d(i) for i in molecules]

Create a [Dropdown](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Dropdown) selector for the 2D structure of the molecules:

In [ ]:
def view_2d(molecule):
    return molecule


molecules_options = [(i, j) for i, j in zip(names, molecules)]

dropdown = Dropdown(
    options=molecules_options, value=molecules[0], description="Molecule:"
)

interact(
    view_2d,
    molecule=dropdown,
)

Create a [Dropdown](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Dropdown) selector for the 3D structure of the molecules:

In [ ]:
def view_3d(molecule):
    view = py3Dmol.view(
        data=Chem.MolToMolBlock(molecule),  # Convert the RDKit molecule for py3Dmol
        style={"stick": {}, "sphere": {"scale": 0.3}}
    )
    view.zoomTo()
    return view


molecules_options = [(i, j) for i, j in zip(names, molecules_3d)]

dropdown = Dropdown(
    options=molecules_options, value=molecules_3d[0], description="Molecule:"
)

interact(
    view_3d,
    molecule=dropdown,
)